In [1]:
import json
import pandas as pd
import requests
from matplotlib import pyplot as plt
# from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split

df = pd.read_csv("dataframes/final_df.csv")
df.head()

/Users/aaronsalazar/LocalDocs/Bologna/ai_industry/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


,id,lat,lon,date,severity,borough_Barking and Dagenham,borough_Barnet,borough_Bexley,borough_Brent,borough_Bromley,...,borough_Southwark,borough_Sutton,borough_Tower Hamlets,borough_Waltham Forest,borough_Wandsworth,casualty_age_0-23,casualty_age_24-30,casualty_age_31-38,casualty_age_39-50,casualty_age_50+
0,345906,51.511963,-0.028211,2019-01-03 01:20:00+00:00,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,345907,51.371636,-0.117621,2019-01-02 22:45:00+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,345908,51.514951,-0.072747,2019-01-02 18:45:00+00:00,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,345909,51.519173,-0.262356,2019-01-03 08:41:00+00:00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,345910,51.565743,-0.136308,2019-01-03 08:37:00+00:00,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
X = df.drop(columns=['severity'])
y = df['severity']  # Assicurati che sia numerico o mappato (1, 2, 3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40500, 42)
(10126, 42)
(40500,)
(10126,)


In [ ]:
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_sample_weight

random_state=42

sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

model = XGBClassifier(random_state=random_state)
model.fit(X_train, y_train, sample_weight=sample_weights)
